In [8]:
# import sql engine, pandas, numpy
import psycopg2
import pandas as pd
import numpy as np

In [5]:
!ls


README.md      Untitled.ipynb db1.sqlite3


In [6]:
# the sqlite database can be converted to use postgresql using 
# pgloader
# first you must have postgresql installed
#then create an empty db to load the database into
#in this instance, I created a 'rover' db

# !pgloader db1.sqlite3 postgresql:///rover

# EXERCISES


# I. Exploring the Database

In [9]:
conn = psycopg2.connect('dbname=rover')
cur = conn.cursor()

### 1. How many users have signed up?

In [26]:
query =  '''
SELECT 
    COUNT(*)
FROM
	people_person;
    '''
cur.execute(query)

df_1= pd.read_sql_query(query,conn)
df_1

,count
0,64393


### 2. How many users signed up prior to 2018-02-03 ?

In [25]:
query =  '''
SELECT 
    COUNT(*)
FROM
	people_person
WHERE date_joined < '2018-02-03'    
    ;
    '''
df_2 = pd.read_sql_query(query,conn)
df_2

,count
0,35826


### 3. What percentage of users have added pets?
#### One thing to note, when dealing with percentages, either cast the numerator to a decimal/numeric, or just make the percentage conversion by using 100.0 (a float) vs 100 (a integer) to keep the precision

In [51]:
query='''
SELECT
    ROUND(count(distinct owner_id)*100.0/(SELECT COUNT(*) FROM people_person),2) as percentage
FROM 
    pets_pet;
'''
df_3= pd.read_sql_query(query,conn)
df_3

# this number gives me the number of users that have registered pets
# now to get a count of the total users, I just divide this number by the total found
# in question 1

,percentage
0,80.43


In [38]:
soln = df_3['count'] / df_1['count']

soln[0].round(4) *100

80.43

### 4. Of those users, how many pets have they added on average?

In [42]:
query ='''
SELECT
    ROUND(avg(count),3) as average_added_pets
FROM
    (
    SELECT 
        COUNT(id) as count, owner_id
    FROM 
        pets_pet
    GROUP BY 
        owner_id) as counts;'''
df_4 = pd.read_sql_query(query,conn)
df_4

,average_added_pets
0,1.501


### 5. What percentage of pets play well with cats?

In [56]:
query = '''
SELECT
ROUND(count(CASE WHEN plays_cats THEN 1 END)*100::decimal/ (SELECT COUNT(*) FROM pets_pet),2) as percentage
FROM pets_pet;
'''
df_5= pd.read_sql_query(query,conn)
df_5

,percentage
0,24.85


 ## III. Recent Daily Booking Rate
 #### The snapshot of this database was taken on 2018-08-02 at midnight and only contains data reflecting events prior to that date. A junior analyst is investigating daily booking rate during the days prior to the snapshot and is concerned about an apparent downward trend. You are tasked with helping them out.
#### First, let's reproduce their results. They tell you that daily booking rate is defined to be the percentage of conversations created each day that eventually book.
#### 1. What is the daily booking rate for each of the 90 days prior to the snapshot? Is there a downward trend?
#### 2. Can you narrate a reason why this trend exists? Is there a reason to be concerned? Please provide additional data and evidence to justify your position.

### 1. What is the daily booking rate for each of the 90 days prior to the snapshot? Is there a downward trend?

In [59]:

# First, lets view the columns of the conversations_conversation table

query = '''
SELECT
    *
FROM
    conversations_conversation
LIMIT 1;
'''
df_3_1 = pd.read_sql_query(query,conn)
df_3_1

,id,start_date,end_date,units,added,booking_total,cancellation_fault,requester_id,service_id,booked_at,cancelled_at
0,1,2018-07-26,2018-07-31,5,2018-07-16 17:17:53.460035+00:00,120,None,64393,4646,None,None


 Looking at the columns and table, to get the count of total conversations, we can use the start_date to indicate each and every conversation that takes place. A pet_owner can start conversations with many different providers, so we want to keep track of that.
 
To find the total of booked services, the simple method would be to just take the total number of 'booked_at' column divided by the total count of the table. But to be more specific, we can also account for when a booked_request is cancelled by also accounting for the 'cancelled_at' column.

In [ ]:
SELECT COUNT(*)

 ## IV. Analyzing Take Rate (IV)

 ## VI. Search Engine Marketing (VI).